<a href="https://colab.research.google.com/github/sushirito/Methylmercury/blob/main/Diffusion_Boltzmann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This code was inspired by: https://github.com/lbm-principles-practice/code/blob/master/chapter9/shanchen.cpp



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tabulate import tabulate
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

# Lattice parameters
nx, ny, nz = 40, 40, 40
nsteps = 100
noutput = 20
nfluids = 1
tau = 1

# Shan-Chen parameters for a single fluid
gA = -0.16
rho0 = 1.0
rhol = 2.1
rhog = 0.15

# Solid-fluid interaction parameters
W = 0.15

# Gravitational force
g_gravity = 9.81e-5

cs_sq = 1/3  # Square of the lattice speed of sound

# Given parameters from the paper
nu = 1/6  # Kinematic viscosity
Lambda_eo = 3/16  # Set according to the paper

# Calculate Lambda_e from kinematic viscosity
Lambda_e = 3 * nu  # Since nu = (1/3) * Lambda_e

# Calculate Lambda_o using the relation with Lambda_eo
Lambda_o = Lambda_eo / Lambda_e

# Calculate lambda_e and lambda_o
lambda_e = -1 / (Lambda_e + 0.5)
lambda_o = -1 / (Lambda_o + 0.5)

# Fixed parameters for D3Q19
npop = 19
cx = np.array([0, 1,-1, 0, 0, 0, 0, 1,-1, 1,-1, 1,-1, 1,-1, 0, 0, 0, 0])
cy = np.array([0, 0, 0, 1,-1, 0, 0, 1, 1,-1,-1, 0, 0, 0, 0, 1,-1, 1,-1])
cz = np.array([0, 0, 0, 0, 0, 1,-1, 0, 0, 0, 0, 1, 1,-1,-1, 1, 1,-1,-1])
weight = np.array([1/3, 1/18, 1/18, 1/18, 1/18, 1/18, 1/18, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36])

# Weights for solid-fluid interactions
t_w = np.array([0, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# Arrays
rho = np.zeros(nx * ny * nz)
press = np.zeros(nx * ny * nz)
ux = np.zeros(nx * ny * nz)
uy = np.zeros(nx * ny * nz)
uz = np.zeros(nx * ny * nz)
Fx = np.zeros(nx * ny * nz)
Fy = np.zeros(nx * ny * nz)
Fz = np.zeros(nx * ny * nz)
feq = np.zeros(npop)
forcing = np.zeros(npop)
f1 = np.zeros((nx * ny * nz, npop))
f2 = np.zeros((nx * ny * nz, npop))

def create_elementary_pattern(Le):
    pattern = np.zeros((Le, Le, Le), dtype=bool)
    radius = Le // 2
    corners = [(0, 0, 0), (0, 0, Le), (0, Le, 0), (Le, 0, 0),
               (0, Le, Le), (Le, 0, Le), (Le, Le, 0), (Le, Le, Le)]

    for corner in corners:
        cx, cy, cz = corner
        x, y, z = np.ogrid[0:Le, 0:Le, 0:Le]
        dist_from_corner = (x - cx)**2 + (y - cy)**2 + (z - cz)**2
        pattern |= (dist_from_corner <= radius**2)

    return pattern

def create_porous_medium(N, Le):
    elementary_pattern = create_elementary_pattern(Le)
    Lm = int(N * Le)
    medium = np.tile(elementary_pattern, (int(np.ceil(N)), int(np.ceil(N)), int(np.ceil(N))))
    return medium[:Lm, :Lm, :Lm]

def calculate_porosity(medium):
    return 1 - np.mean(medium)

# Parameters
Le = 10
N = 4

# Create porous medium
solid = create_porous_medium(N, Le)

# Calculate porosity
porosity = calculate_porosity(solid)
print(f"Porosity: {porosity:.3f}")

def equilibrium(k):
    dens = rho[k]
    vx, vy, vz = ux[k], uy[k], uz[k]

    feq = np.zeros(npop)

    # e_0^+ (q = 0)
    feq[0] = dens - np.sum([weight[q] * cs_sq * dens for q in range(1, npop)])

    # e_q^+ (q = 1, ..., Q-1)
    for q in range(1, npop):
        feq[q] = weight[q] * cs_sq * dens

    # e_q^- (q = 1, ..., Q-1)
    for q in range(1, npop):
        feq[q] += weight[q] * (cx[q]*vx + cy[q]*vy + cz[q]*vz) * dens

    return feq

def compute_moments():
    global rho, ux, uy, uz
    rho[:] = np.sum(f1, axis=1)
    epsilon = 1e-10  # A small constant to prevent division by zero
    for i in range(npop):
        ux += cx[i] * f1[:, i]
        uy += cy[i] * f1[:, i]
        uz += cz[i] * f1[:, i]
    ux /= (rho + epsilon)
    uy /= (rho + epsilon)
    uz /= (rho + epsilon)

def psi(dens):
    return rho0 * (1.0 - np.exp(np.clip(-dens / rho0, -50, 0)))

def compute_sc_forces():
    global Fx, Fy, Fz
    for z in range(nz):
        for y in range(ny):
            for x in range(nx):
                k = z * nx * ny + y * nx + x
                Fx[k] = Fy[k] = Fz[k] = 0.0
                psiloc = psi(rho[k])
                for i in range(1, npop):
                    x2 = (x + cx[i] + nx) % nx
                    y2 = (y + cy[i] + ny) % ny
                    z2 = (z + cz[i] + nz) % nz
                    k2 = z2 * nx * ny + y2 * nx + x2
                    psinb = psi(rho[k2])
                    force = gA * weight[i] * psiloc * psinb
                    Fx[k] -= np.clip(force * cx[i], -1e3, 1e3)
                    Fy[k] -= np.clip(force * cy[i], -1e3, 1e3)
                    Fz[k] -= np.clip(force * cz[i], -1e3, 1e3)
                    if solid[z2, y2, x2] == 1:
                        Fx[k] -= np.clip(W * weight[i] * cx[i] * psiloc, -1e3, 1e3)
                        Fy[k] -= np.clip(W * weight[i] * cy[i] * psiloc, -1e3, 1e3)
                        Fz[k] -= np.clip(W * weight[i] * cz[i] * psiloc, -1e3, 1e3)
                Fz[k] -= np.clip(rho[k] * g_gravity, -1e3, 1e3)

def collision_and_streaming():
    global f1, f2
    for z in range(nz):
        for y in range(ny):
            for x in range(nx):
                k = z * nx * ny + y * nx + x
                feq = equilibrium(k)
                for i in range(npop):
                    f_plus = 0.5 * (f1[k, i] + f1[k, npop-1-i])
                    f_minus = 0.5 * (f1[k, i] - f1[k, npop-1-i])
                    n_plus = np.clip(f_plus - feq[i], -1e3, 1e3)
                    n_minus = np.clip(f_minus - 0.5 * weight[i] * (ux[k]*cx[i] + uy[k]*cy[i] + uz[k]*cz[i]), -1e3, 1e3)

                    x2 = (x + cx[i] + nx) % nx
                    y2 = (y + cy[i] + ny) % ny
                    z2 = (z + cz[i] + nz) % nz
                    k2 = z2 * nx * ny + y2 * nx + x2

                    forcing = np.clip(weight[i] * (cx[i]*Fx[k] + cy[i]*Fy[k] + cz[i]*Fz[k]), -1e3, 1e3)
                    f2[k2, i] = np.clip(f1[k, i] + lambda_e * n_plus + lambda_o * n_minus + forcing, 0, 1e6)

    f1, f2 = f2, f1

def bounce_back_boundaries():
    for i in range(1, npop):
        f1[solid.flatten() == 1, i] = f1[solid.flatten() == 1, npop - 1 - i]

def initialisation():
    global f1, f2, rho
    rho = np.zeros(nx * ny * nz)
    rho_i = np.random.uniform(0.5, 1.5)  # Narrower range
    for k in range(nx * ny * nz):
        if solid.flatten()[k] == 0:
            rho[k] = np.clip(rho_i, 0.1, 2.0)  # Clip to ensure reasonable values
        else:
            rho[k] = 0

    for k in range(nx * ny * nz):
        ux[k] = uy[k] = uz[k] = 0.0
        feq = equilibrium(k)
        f1[k] = np.clip(feq, 0, 1e6)  # Ensure non-negative values
        f2[k] = f1[k]

def calculate_saturation():
    rho_liq_th = 2.42
    rho_gas_th = 0.03

    # Calculate average density as a possible threshold
    rho_threshold = (rho_liq_th + rho_gas_th) / 2

    # Calculate number of liquid nodes and total fluid nodes
    liquid_nodes = np.sum((rho > rho_threshold) & (solid.flatten() == 0))
    total_fluid_nodes = np.sum(solid.flatten() == 0)

    # Print for visualization
    print(f"Liquid Nodes: {liquid_nodes}")
    print(f"Total Fluid Nodes: {total_fluid_nodes}")

    # Use theoretical densities to calculate saturation
    saturation = (np.mean(rho) - rho_gas_th) / (rho_liq_th - rho_gas_th)
    return saturation

output_dir = '/content/lbm_output'
os.makedirs(output_dir, exist_ok=True)

def save_html(fig, filename):
    filepath = os.path.join(output_dir, filename)
    fig.write_html(filepath)
    print(f"Saved: {filepath}")

def save_png(fig, filename):
    filepath = os.path.join(output_dir, filename)
    fig.savefig(filepath)
    plt.close(fig)
    print(f"Saved: {filepath}")

def density_plot_3d(rho, step):
    x, y, z = np.meshgrid(np.arange(nx), np.arange(ny), np.arange(nz))
    scalar_field = rho.reshape((nz, ny, nx)).flatten()

    print(f"Min density: {np.min(scalar_field)}")
    print(f"Max density: {np.max(scalar_field)}")
    print(f"Mean density: {np.mean(scalar_field)}")

    fig = go.Figure(data=go.Scatter3d(
        x=x.flatten(),
        y=y.flatten(),
        z=z.flatten(),
        mode='markers',
        marker=dict(
            size=3,
            color=scalar_field,
            colorscale='Viridis',
            opacity=0.5,
            colorbar=dict(title='Density')
        )
    ))

    fig.update_layout(
        title=f'Density Distribution at Step {step}',
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )

    fig.show()
    save_html(fig, f'density_plot_3d_{step:05d}.html')

def velocity_magnitude_plot_3d(ux, uy, uz, step):
    x, y, z = np.meshgrid(np.arange(nx), np.arange(ny), np.arange(nz))
    velocity_magnitude = np.sqrt(ux**2 + uy**2 + uz**2).reshape((nz, ny, nx)).flatten()

    fig = go.Figure(data=go.Scatter3d(
        x=x.flatten(),
        y=y.flatten(),
        z=z.flatten(),
        mode='markers',
        marker=dict(
            size=3,
            color=velocity_magnitude,
            colorscale='Plasma',
            opacity=0.5,
            colorbar=dict(title='Velocity Magnitude')
        )
    ))

    fig.update_layout(
        title=f'Velocity Magnitude at Step {step}',
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )

    fig.show()
    save_html(fig, f'velocity_magnitude_3d_{step:05d}.html')

def velocity_streamplot_3d(ux, uy, uz, step):
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    x, y, z = np.meshgrid(np.arange(nx), np.arange(ny), np.arange(nz))

    skip = (slice(None, None, 4), slice(None, None, 4), slice(None, None, 4))
    ax.quiver(x[skip], y[skip], z[skip],
              ux.reshape((nz, ny, nx))[skip],
              uy.reshape((nz, ny, nx))[skip],
              uz.reshape((nz, ny, nx))[skip],
              length=2, normalize=True)

    ax.set_title(f'Velocity Streamlines at Step {step}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()
    save_png(fig, f'velocity_streamlines_3d_{step:05d}.png')

def main():
    initialisation()
    rho_history = []
    saturation_history = []
    phase_separation_table = []

    for step in range(nsteps + 1):
        compute_moments()
        compute_sc_forces()
        collision_and_streaming()
        bounce_back_boundaries()

        # Check for NaN or inf values
        if np.any(np.isnan(rho)) or np.any(np.isinf(rho)):
            print(f"Error: NaN or inf values detected in density at step {step}")
            break

        if step % noutput == 0:
            print(f"Running time step {step}")
            print(f"Min density: {np.min(rho)}")
            print(f"Max density: {np.max(rho)}")
            print(f"Mean density: {np.mean(rho)}")

            # Store density history
            rho_history.append(rho.copy())

            # Create and save visualizations
            density_plot_3d(rho, step)
            velocity_magnitude_plot_3d(ux, uy, uz, step)
            velocity_streamplot_3d(ux, uy, uz, step)

            # Calculate and store saturation
            saturation = calculate_saturation()
            saturation_history.append(saturation)

            # Calculate phase separation data
            rho_i = np.mean(rho)
            rho_liq = np.max(rho)
            rho_gas = np.min(rho)
            s_th = (rho_i - rho_gas) / (rho_liq - rho_gas)
            s_cal = saturation

            phase_separation_table.append([rho_i, s_th, rho_liq, rho_gas, s_cal])

    # Print phase separation table
    print("\nPhase Separation Table:")
    headers = ["ρi", "S_th", "ρ_liq^cal", "ρ_gas^cal", "S_cal"]
    print(tabulate(phase_separation_table, headers=headers, floatfmt=".3f"))

    # Plot saturation evolution
    plt.figure(figsize=(10, 6))
    plt.plot(range(0, nsteps+1, noutput), saturation_history)
    plt.xlabel('Time Step')
    plt.ylabel('Saturation')
    plt.title('Saturation Evolution')
    plt.savefig(os.path.join(output_dir, 'saturation_evolution.png'))
    plt.close()

    print(f"All output files have been saved in: {output_dir}")

if __name__ == "__main__":
    main()

In [20]:
import numpy as np
import plotly.graph_objects as go
import os
import matplotlib.pyplot as plt

# Lattice parameters
nx, ny, nz = 20, 20, 20  # Domain size
nsteps = 500  # Number of time steps
noutput = 100  # Output frequency

# D3Q7 model parameters
npop = 7  # Number of populations
cx = np.array([0, 1, -1, 0, 0, 0, 0])
cy = np.array([0, 0, 0, 1, -1, 0, 0])
cz = np.array([0, 0, 0, 0, 0, 1, -1])
weight = np.array([1/4, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8])  # Weights for D3Q7

# Diffusion parameters
D0 = 0.5  # Molecular diffusion coefficient [l.u.]^2 / [t.u.]
Lambda_eo = 1/32  # As given in the paper
cs_sq = 1/3  # Square of the lattice speed of sound

lambda_o = -2*cs_sq/(2*D0+cs_sq)
Lambda_o = -(1/2 + 1/lambda_o)
Lambda_e = Lambda_eo/Lambda_o
lambda_e = -2/(2*Lambda_e+1)

# Initialize arrays
rho = np.zeros(nx * ny * nz)  # Density (concentration)
f1 = np.zeros((nx * ny * nz, npop))  # Distribution functions
f2 = np.zeros((nx * ny * nz, npop))  # Temporary array for streaming

def create_elementary_pattern(Le):
    pattern = np.zeros((Le, Le, Le), dtype=bool)
    radius = Le // 2
    corners = [(0, 0, 0), (0, 0, Le), (0, Le, 0), (Le, 0, 0),
               (0, Le, Le), (Le, 0, Le), (Le, Le, 0), (Le, Le, Le)]

    for corner in corners:
        cx, cy, cz = corner
        x, y, z = np.ogrid[0:Le, 0:Le, 0:Le]
        dist_from_corner = (x - cx)**2 + (y - cy)**2 + (z - cz)**2
        pattern |= (dist_from_corner <= radius**2)

    return pattern

def create_porous_medium(N, Le):
    elementary_pattern = create_elementary_pattern(Le)
    Lm = int(N * Le)
    medium = np.tile(elementary_pattern, (int(np.ceil(N)), int(np.ceil(N)), int(np.ceil(N))))
    return medium[:Lm, :Lm, :Lm]

def calculate_porosity(medium):
    return 1 - np.mean(medium)

# Parameters
Le = 10
N = 2

# Create porous medium
solid = create_porous_medium(N, Le)

# Calculate porosity
porosity = calculate_porosity(solid)
print(f"Porosity: {porosity:.3f}")

# Equilibrium functions
def equilibrium(rho_local):
    feq = np.zeros(npop)
    feq[0] = weight[0] * rho_local
    for q in range(1, npop):
        feq[q] = weight[q] * rho_local
    return feq

# Compute moments (simplified for diffusion)
def compute_moments():
    global rho
    rho[:] = np.sum(f1, axis=1)

# Collision and streaming step
def collision_and_streaming():
    global f1, f2
    for z in range(nz):
        for y in range(ny):
            for x in range(nx):
                k = z * nx * ny + y * nx + x
                if solid[z, y, x] == 0:  # Fluid node
                    feq = equilibrium(rho[k])
                    for i in range(npop):
                        opp = npop-1-i  # Index of the opposite direction
                        f_plus = 0.5 * (f1[k, i] + f1[k, opp])
                        f_minus = 0.5 * (f1[k, i] - f1[k, opp])

                        e_plus = feq[i]
                        e_minus = 0  # For diffusion, e⁻q = 0 as J = 0

                        n_plus = f_plus - e_plus
                        n_minus = f_minus - e_minus

                        x2 = (x + cx[i] + nx) % nx
                        y2 = (y + cy[i] + ny) % ny
                        z2 = (z + cz[i] + nz) % nz
                        k2 = z2 * nx * ny + y2 * nx + x2

                        f2[k2, i] = f1[k, i] + lambda_e * n_plus + lambda_o * n_minus
                else:  # Solid node
                    for i in range(npop):
                        x2 = (x - cx[i] + nx) % nx
                        y2 = (y - cy[i] + ny) % ny
                        z2 = (z - cz[i] + nz) % nz
                        k2 = z2 * nx * ny + y2 * nx + x2
                        f2[k2, npop-1-i] = f1[k, i]  # Bounce-back

    f1, f2 = f2, f1
# Initialization
def initialize_diffusion():
    global f1, f2, rho

    # Initialize concentration
    rho_init = 0.5  # Initial concentration
    rho = np.where(solid.flatten() == 0, rho_init, 0)

    # Initialize distribution functions
    for k in range(nx * ny * nz):
        if solid.flatten()[k] == 0:
            feq = equilibrium(rho[k])
            f1[k] = feq
            f2[k] = feq

# Assuming output_dir is defined as the directory where you want to save the plots
output_dir = 'output_plots'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def save_html(fig, filename):
    filepath = os.path.join(output_dir, filename)
    fig.write_html(filepath)
    print(f"Saved: {filepath}")

def save_png(fig, filename):
    filepath = os.path.join(output_dir, filename)
    fig.savefig(filepath)
    plt.close(fig)
    print(f"Saved: {filepath}")

def concentration_plot_3d(rho, step):
    # Creating meshgrid for 3D plot
    z, y, x = np.meshgrid(np.arange(nz), np.arange(ny), np.arange(nx))
    scalar_field = rho.reshape((nz, ny, nx)).flatten()

    # Print density information for debugging
    print(f"Min concentration: {np.min(scalar_field)}")
    print(f"Max concentration: {np.max(scalar_field)}")
    print(f"Mean concentration: {np.mean(scalar_field)}")

    # Create a 3D scatter plot
    fig = go.Figure(data=go.Scatter3d(
        x=x.flatten(),
        y=y.flatten(),
        z=z.flatten(),
        mode='markers',
        marker=dict(
            size=3,
            color=scalar_field,
            colorscale='Viridis',  # Change color scale if desired
            opacity=0.5,
            colorbar=dict(title='Concentration')
        )
    ))

    # Update layout for better visualization
    fig.update_layout(
        title=f'Concentration Distribution at Step {step}',
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )

    # Show the figure
    fig.show()

    # Save the plot as HTML
    save_html(fig, f'concentration_plot_3d_{step:05d}.html')

# Run the main loop and generate plots at each output step
def main_loop():
    initial_mass = np.sum(rho)
    for step in range(nsteps):
        compute_moments()
        collision_and_streaming()

        if step % noutput == 0:
            current_mass = np.sum(rho)
            mass_change = (current_mass - initial_mass) / initial_mass
            print(f"Step {step}: Average concentration = {np.mean(rho[solid.flatten() == 0])}")
            print(f"Mass change: {mass_change:.6%}")
            concentration_plot_3d(rho, step)

    final_mass = np.sum(rho)
    total_mass_change = (final_mass - initial_mass) / initial_mass
    print(f"Total mass change: {total_mass_change:.6%}")

# Initialize and run simulation
initialize_diffusion()
main_loop()

print("Simulation complete.")

Porosity: 0.488
Step 0: Average concentration = 0.5
Mass change: 0.000000%
Min concentration: 0.0
Max concentration: 0.5
Mean concentration: 0.244


Saved: output_plots/concentration_plot_3d_00000.html
Step 100: Average concentration = 0.3899924710285233
Mass change: -17.249315%
Min concentration: -0.015021450989312776
Max concentration: 0.9431468292572104
Mean concentration: 0.2019116708126785


Saved: output_plots/concentration_plot_3d_00100.html
Step 200: Average concentration = 0.389698261839901
Mass change: -17.309900%
Min concentration: -0.015000022939699494
Max concentration: 0.9427104911439315
Mean concentration: 0.20176384317323007


Saved: output_plots/concentration_plot_3d_00200.html
Step 300: Average concentration = 0.38971643922291416
Mass change: -17.306075%
Min concentration: -0.015001727856720874
Max concentration: 0.942250666886387
Mean concentration: 0.20177317697692262


Saved: output_plots/concentration_plot_3d_00300.html
Step 400: Average concentration = 0.3897127575520052
Mass change: -17.306815%
Min concentration: -0.015001690636331902
Max concentration: 0.9423299222663235
Mean concentration: 0.20177137035414167


Saved: output_plots/concentration_plot_3d_00400.html
Total mass change: -17.306795%
Simulation complete.
